In [1]:
%run preamble.ipynb

The preamble handles a few imports and defines the following functions:

printLoss(y_tr,p_tr,y_te,p_te,loss=log_loss)
createDataSet(predictions, group_encoder, device_ids)
getBestPrediction(data,var='device_id')


A few Debug functions


In [12]:
data = pd.read_csv("files/finalSets/apps_full.csv", nrows=2)

In [13]:
data.head()

,event_id,app_id,is_active,nCats,UniqueLabel,device_id,hour,usageDay,isTrain,group,nActiveApps,nInstallApps,phone_brand,device_model,nEvts
0,2,5927333115845830913,1,4.0,cat_Family,-6401643145415154744,0,3,0,none,6,19,??,Galaxy Grand Prime,73
1,2,-5720078949152207372,0,4.0,cat_Productivity,-6401643145415154744,0,3,0,none,6,19,??,Galaxy Grand Prime,73


## Variables to use in the model
* categories: [`event_id`, `UniqueLabel`, `usageDay`, `phone_brand`, `device_model`, `nCats`]
   * this results in memory issues (too many dimensions)
   * drop `nCats`, `phone_brand`
* `hour`
* `nInstallApps`
* `nActiveApps`
* `nEvts`
* `is_active`


In [4]:
cols = ["event_id","is_active","UniqueLabel", "device_id","hour","usageDay","isTrain","group"
        ,"nActiveApps","nInstallApps","device_model","nEvts"]

In [5]:
data = pd.read_csv("files/finalSets/apps_full.csv", usecols=cols)

In [ ]:
scaler_nAct = StandardScaler()
scaler_nInst = StandardScaler()
scaler_hour = StandardScaler()
scaler_nevts = StandardScaler()
data["nActiveApps"] = scaler_nAct.fit_transform(data.nActiveApps.reshape(-1, 1))
data["nEvts"] = scaler_nevts.fit_transform(data.nEvts.reshape(-1, 1))
data["nInstallApps"] = scaler_nInst.fit_transform(data.nInstallApps.reshape(-1, 1))
data["hour"] = scaler_hour.fit_transform(data.hour.reshape(-1,1))

In [6]:
#enc_brand = LabelEncoder()
enc_device = LabelEncoder()
enc_group = LabelEncoder()
enc_label = LabelEncoder()
#data["phone_brand"] = enc_brand.fit_transform(data.phone_brand)
#print("brand done.")
data["device_model"] = enc_device.fit_transform(data.device_model)
print("model done.")
data["group"] = enc_group.fit_transform(data.group)
print("group done.")

model done.
group done.


In [7]:
data = data.dropna()

In [8]:
data["UniqueLabel"] = enc_label.fit_transform(data.UniqueLabel)
print("label done.")

label done.


In [7]:
data.drop(["isTrain","device_id","group"], axis=1).columns

Index(['event_id', 'is_active', 'UniqueLabel', 'hour', 'usageDay',
       'nActiveApps', 'nInstallApps', 'device_model', 'nEvts'],
      dtype='object')

In [7]:
lr_enc = joblib.load("OneHotEncoder.pkl")

In [41]:
#lr_enc = OneHotEncoder()#categorical_features=[0,2,5])
#lr_enc.fit(data.drop(["isTrain","device_id","group"],axis=1))
#joblib.dump(lr_enc, "OneHotEncoder.pkl", compress=3)

['OneHotEncoder.pkl']

In [9]:
drop_list = ["isTrain"]
train = data[data.isTrain==1].drop( drop_list, axis=1)
true_classes = train.group
test = data[data.isTrain==0].drop( drop_list+["group"], axis=1)

In [19]:
tt = test.drop(["event_id","is_active","UniqueLabel","hour"
                ,"usageDay","nActiveApps","device_model","nEvts"],axis=1)

In [22]:
tt.drop("nInstallApps",axis=1).to_csv("files/app_test_devid.csv",index=False)

In [127]:
x_train, x_test, y_train, y_test = train_test_split(train
                                                    , true_classes
                                                    , test_size=0.5
                                                    , random_state=999)

In [ ]:
train_drop = ["group","device_id"]

In [12]:
lr = LogisticRegression(penalty='l2'
                        , C=0.4 
                        , tol=0.001 # was 0.0001
                        , solver= "lbfgs"#"liblinear" #''
                        , max_iter=100
                        , warm_start=True
                        , multi_class='multinomial'
                        , verbose = 2
                        , n_jobs=2
                       )

In [ ]:
s = time.time()
lr.fit(lr_enc.transform(x_train.drop(train_drop,axis=1)),y_train)
print(" done in ", (time.time()-s)/60.0, " minutes.")

In [28]:
joblib.dump(lr, "trainedModels/lr_App_C001_lowTol.pkl", compress=3)

['trainedModels/lr_App_C001_lowTol.pkl']

In [29]:
probs_lr_train = lr.predict_proba(lr_enc.transform(x_train.drop(train_drop,axis=1)))
probs_lr_test = lr.predict_proba(lr_enc.transform(x_test.drop(train_drop,axis=1)))

In [30]:
printLoss(y_train, probs_lr_train, y_test, probs_lr_test)

Test MVA predictions on test and training set:

Log loss on training set:  1.4357604485
Log loss on test set:  1.43789594807


* loss on highC04: 1.952 (on test, not overtrained). 
* loss on lbfgs c001 (val): 1.43 (not overtrained). best
* loss on test with above and reduced tolerance: 1.43 (same...)

### Predict test sample with LR model

In [34]:
pred_test = lr.predict_proba(lr_enc.transform(test.drop("device_id",axis=1)))

In [35]:
pred_test = createDataSet(pred_test, enc_group, test.device_id.values)
pred_test.to_csv("finalOutputs/lr_App_c001_lowTol.csv", index=False)

In [22]:
#av_pred_test = averagePredictions(pred_test, test.device_id.values)


In [25]:
#av_pred_test.to_csv("finalOutputs/lr_App_C04lin.csv", index=False)

# Random Forest

In [11]:
from sklearn.ensemble import RandomForestClassifier

In [12]:
rf = RandomForestClassifier(n_estimators=200
                            , criterion='entropy'
                            , min_samples_split=20
                            , min_samples_leaf=1
                            , max_leaf_nodes=None
                            , n_jobs=3
                            , random_state=666
                            , verbose=3
                            #, warm_start=True
                           )

In [13]:
s=time.time()/60.0
rf.fit(x_train.drop(train_drop,axis=1),y_train)
print("rf done in ", (time.time()/60.0)-s, " minutes.")

building tree 1 of 200building tree 2 of 200
building tree 3 of 200

building tree 4 of 200
building tree 5 of 200
building tree 6 of 200
building tree 7 of 200
building tree 8 of 200
building tree 9 of 200
building tree 10 of 200
building tree 11 of 200
building tree 12 of 200
building tree 13 of 200
building tree 14 of 200
building tree 15 of 200
building tree 16 of 200
building tree 17 of 200
building tree 18 of 200
building tree 19 of 200
building tree 20 of 200
building tree 21 of 200
building tree 22 of 200
building tree 23 of 200
building tree 24 of 200
building tree 25 of 200
building tree 26 of 200
building tree 27 of 200
building tree 28 of 200
building tree 29 of 200


[Parallel(n_jobs=3)]: Done  26 tasks      | elapsed:  9.3min


building tree 30 of 200
building tree 31 of 200
building tree 32 of 200
building tree 33 of 200
building tree 34 of 200
building tree 35 of 200
building tree 36 of 200
building tree 37 of 200
building tree 38 of 200
building tree 39 of 200
building tree 40 of 200
building tree 41 of 200
building tree 42 of 200
building tree 43 of 200
building tree 44 of 200
building tree 45 of 200
building tree 46 of 200
building tree 47 of 200
building tree 48 of 200
building tree 49 of 200
building tree 50 of 200
building tree 51 of 200
building tree 52 of 200
building tree 53 of 200
building tree 54 of 200
building tree 55 of 200
building tree 56 of 200
building tree 57 of 200
building tree 58 of 200
building tree 59 of 200
building tree 60 of 200
building tree 61 of 200
building tree 62 of 200
building tree 63 of 200
building tree 64 of 200
building tree 65 of 200
building tree 66 of 200
building tree 67 of 200
building tree 68 of 200
building tree 69 of 200
building tree 70 of 200
building tree 71

[Parallel(n_jobs=3)]: Done 122 tasks      | elapsed: 41.4min


building tree 126 of 200
building tree 127 of 200
building tree 128 of 200
building tree 129 of 200
building tree 130 of 200
building tree 131 of 200
building tree 132 of 200
building tree 133 of 200
building tree 134 of 200
building tree 135 of 200
building tree 136 of 200
building tree 137 of 200
building tree 138 of 200
building tree 139 of 200
building tree 140 of 200
building tree 141 of 200
building tree 142 of 200
building tree 143 of 200
building tree 144 of 200
building tree 145 of 200
building tree 146 of 200
building tree 147 of 200
building tree 148 of 200
building tree 149 of 200
building tree 150 of 200
building tree 151 of 200
building tree 152 of 200
building tree 153 of 200
building tree 154 of 200
building tree 155 of 200
building tree 156 of 200
building tree 157 of 200
building tree 158 of 200
building tree 159 of 200
building tree 160 of 200
building tree 161 of 200
building tree 162 of 200
building tree 163 of 200
building tree 164 of 200
building tree 165 of 200


[Parallel(n_jobs=3)]: Done 200 out of 200 | elapsed: 72.6min finished


In [16]:
joblib.dump(rf, "trainedModels/rf_app_200_sample20.pkl", compress=1)

ValueError: I/O operation on closed file.

In [129]:
probs_rf_train = rf.predict_proba(x_train.drop(train_drop,axis=1))
probs_rf_val = rf.predict_proba(x_val.drop(train_drop,axis=1))
printLoss(y_train, probs_rf_train, y_val, probs_rf_val)

SyntaxError: invalid or missing encoding declaration for 'C:\\Users\\maxxo_000\\Anaconda3\\envs\\jupnote\\lib\\site-packages\\sklearn\\tree\\_tree.pyd' (<string>)

In [ ]:
probs_rf_train = createDataSet(probs_rf_train, enc_group, x_train.device_id.values)
probs_rf_val = createDataSet(probs_rf_val, enc_group, x_val.device_id.values)

In [ ]:
probs_rf_train = getBestPrediction(probs_rf_train)
probs_rf_val = getBestPrediction(probs_rf_val)
printLoss(y_train.iloc[probs_rf_train.index.values]
          , probs_rf_train.drop("device_id",axis=1).as_matrix()
          , y_val.iloc[probs_rf_val.index.values]
          , probs_rf_val.drop("device_id",axis=1).as_matrix())

In [43]:
len(test)

65933.94158075601

In [41]:
29*661613

19186777

In [110]:
slices = []
step = 500000
n =int(19186777/step)

for i in range(n):
    slices.append([i*step,(i+1)*step])

In [112]:
slices.append([19000000,19186777])

In [114]:
probs_rf_test = pd.DataFrame()

In [115]:
rf.verbose = 0

In [116]:
for i, s in enumerate(slices):
    probs_rf_test = pd.concat([probs_rf_test,
                               pd.DataFrame(rf.predict_proba(test[s[0]:s[1]]\
                                                .drop("device_id",axis=1)))]
                              )
    if i % 5 == 0 and i>0:
        print((i+1)*step,"/", (len(slices)+1)*step," events processed")

3000000 / 20000000  events processed
5500000 / 20000000  events processed
8000000 / 20000000  events processed
10500000 / 20000000  events processed
13000000 / 20000000  events processed
15500000 / 20000000  events processed
18000000 / 20000000  events processed


In [128]:
probs_rf_test.to_csv("finalOutputs/rf_app_200_split20.csv", index=False)

# Neural Net

In [45]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout

Using Theano backend.


In [63]:
true_classes_nn = x_train.group.values

In [64]:
# we would like to use the categorical_crossentropy (multiclass logloss)
# so let's convert classes to categroies
from keras.utils.np_utils import to_categorical
true_classes_nn = to_categorical(true_classes_nn)
# true_classes now is a binary matrix

In [51]:
len(x_train.drop(train_drop,axis=1).columns)

7

In [52]:
# in keras we need to build models.
# we build our own sequential model
model = Sequential()

# first we add a dense layer (std NN layer)
# we need an output of 12 dimensions
model.add(Dense(output_dim=30, input_dim=7))
model.add(Activation("tanh")) # no real motivation for relu here
model.add(Dropout(0.4))
model.add(Dense(output_dim=30, activation="tanh"))
model.add(Dropout(0.4))
model.add(Dense(output_dim=23, activation="tanh"))
model.add(Dropout(0.4))
model.add(Dense(output_dim=17, activation="tanh"))
model.add(Dropout(0.4))
model.add(Dense(output_dim=12))
model.add(Activation("softmax"))

In [53]:
# now we need to configure the learning process
model.compile(loss='categorical_crossentropy'
              ,optimizer = "adadelta" 
              #optimizer='adam'
              , metrics=['accuracy'])

In [ ]:
s = time.time()
model.fit(x_train.drop(train_drop,axis=1).as_matrix(),true_classes_nn
         , verbose=1,nb_epoch=20)
print("NN trained in ", (time.time()-s)/60.0, " minutes")

In [ ]:
score = model.evaluate(in_data.as_matrix(), true_classes_nn)